In [24]:
from dotenv import load_dotenv
import gradio as gr
from pypdf import PdfReader
from openai import OpenAI
import os


In [25]:
load_dotenv(override=True)

openai = OpenAI()

gemini = OpenAI(api_key=os.getenv("GOOGLE_API_KEY"), base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

reader = PdfReader("E:/Mis Carpetas/Documentos/Python/Udemy_Ing_Agentes/projects/agents/1_foundations/me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("E:/Mis Carpetas/Documentos/Python/Udemy_Ing_Agentes/projects/agents/1_foundations/me/summary.txt", mode="r",encoding="utf-8") as f:
    summary = f.read()

name = "Juan Gabriel Gomila Salas"

system_prompt = f"Estás actuando como {name}. Estás respondiendo preguntas en el sitio web de {name}, en particular preguntas relacionadas con la carrera, la trayectoria, las habilidades y la experiencia de {name}. \
Tu responsabilidad es representar a {name} en las interacciones en el sitio web con la mayor fidelidad posible. \
Se te proporciona un resumen de la trayectoria y el perfil de LinkedIn de {name} que puedes usar para responder preguntas. \
Sé profesional y atractivo, como si hablaras con un cliente potencial o un futuro empleador que haya visitado el sitio web. \
Si no sabes la respuesta, dilo."

system_prompt += f"\n\n## Resumen:\n{summary}\n\n## Perfil de LinkedIn:\n{linkedin}\n\n"
system_prompt += f"En este contexto, charla con el usuario, utilizando siempre el personaje de {name}."


In [20]:
def chat(message, history):
    messages=[{"role":"system", "content":system_prompt}] + history + [{"role":"user", "content":message}]
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages = messages
    )
    return response.choices[0].message.content

In [21]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [22]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

evaluator_system_prompt = f"Usted es un evaluador que decide si una respuesta a una pregunta es aceptable. \
Se le presenta una conversación entre un usuario y un agente. Su tarea es decidir si la última respuesta del agente es de calidad aceptable. \
El agente desempeña el papel de {name} y representa a {name} en su sitio web. \
Se le ha indicado que sea profesional y atractivo, como si hablara con un cliente potencial o un futuro empleador que haya visitado el sitio web. \
Se le ha proporcionado contexto sobre {name} en forma de resumen y datos de LinkedIn. Aquí está la información:"

evaluator_system_prompt += f"\n\n## Resumen:\n{summary}\n\n## Perfil de LinkedIn:\n{linkedin}\n\n"
evaluator_system_prompt += f"Con este contexto, por favor, evalúe la última respuesta, indicando si es aceptable y sus comentarios."

In [23]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Aquí está la conversación entre el usuario y el agente: \n\n{history}\n\n"
    user_prompt += f"Aquí está el último mensaje del usuario: \n\n{message}\n\n"
    user_prompt += f"Aquí está la última respuesta del agente: \n\n{reply}\n\n"
    user_prompt += f"Por favor, evalúe la respuesta, indicando si es aceptable y sus comentarios."
    return user_prompt

In [39]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role":"system", "content":evaluator_system_prompt}] + [{"role":"user", "content":evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Respuesta anterior rechazada\nAcabas de intentar responder, pero el control de calidad rechazó tu respuesta.\n"
    updated_system_prompt += f"## Has intentado responder:\n{reply}\n\n"
    updated_system_prompt += f"## Razón del rechazo:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [44]:
def chat(message, history):
    if "instrumento" in message:
        system = system_prompt + "\n\nToda tu respuesta debe estar en el latín de los cerdos traducido al español -\
              Es obligatorio que respondas únicamente y en su totalidad en el latín de los cerdos traducido al español."
    else:
        system = system_prompt
    
    messages = [{"role":"system", "content":system}] + history + [{"role":"user", "content":message}]
    response = openai.chat.completions.create(model="gpt-4o-mini",messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
         print("Has pasado la evaluación - devolviendo respuesta")
    else:
        print("Has fallado la evaluación - reintentando")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply
    

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


Has pasado la evaluación - devolviendo respuesta
Has pasado la evaluación - devolviendo respuesta
